## SQL ##

*En esta fase del proceso de ETL  vamos a crear paso a paso una base de datos en sql desde python, para despúes crear las tablas con sus PK y sus FK.*

*Una vez todo esto este creado procedemos a poblarla desde python con nuestro df limpio y actualizado.*

Este es el codigo usado en sql para crear nuestra base de datos. *Este codigo se puede usar para crear la base de datos directamente desde SQL,pero vamos a crearla desde este entorno*

In [1]:
import pandas as pd
import mysql.connector
import numpy as np

Este es el codigo para crear desde python la base de datos en sql.

*Correr solo una vez*

In [2]:
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"#Aqui teneis que poner vuestra contraeña.
)

cursor = db.cursor()

# Crear la base de datos si no existe
cursor.execute('CREATE DATABASE IF NOT EXISTS AIRBNB')

# Conectar a la base de datos recién creada
db.database = 'AIRBNB'

# Crear la tabla Hosting
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Hosting (
        record_id BIGINT,
        titles TEXT,
        property_types TEXT,
        host_name TEXT,
        PRIMARY KEY (record_id)               
    );
''')

# Crear la tabla Description_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Description_ (
        id BIGINT,
        prices_per_night INT,
        check_in_hour TIME,
        check_out_hour TIME,
        total_hours_checkin INT,
        cleaning_fee INT,
        maximum_guests INT,
        location TEXT,
        camas INT,
        baños INT,
        dormitorios INT,
        FOREIGN KEY (id) REFERENCES Hosting(record_id)
    );
''')

# Crear la tabla Ratings
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Ratings (
        record_id BIGINT,
        ratings FLOAT,
        num_reviews FLOAT,
        FOREIGN KEY (record_id) REFERENCES Hosting(record_id)
    );
''')

#Creamos la tabla de Services_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Services_ (
        service_id INT,
        service TEXT,
        PRIMARY KEY (service_id)
   );
''')
#Creamos la tabla de Service_ids
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Services_ids (
        service_id INT,
        record_id BIGINT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (record_id) REFERENCES Hosting (record_id)
);
''')   
#Creamos la tabla de Category
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Category (
        category_id INT,
        category TEXT,
        PRIMARY KEY (category_id)
 );
''')

#Creamos la tabla de Category_ids
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Category_ids (
        service_id INT,
        category_id INT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (category_id) REFERENCES Category(category_id)
);
''')   

# Cerrar el cursor y la conexión
cursor.close()
db.close()


Este es el codigo para la subida de nuestro df a la base de datos

In [2]:
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_final_cleaned.csv')
df_category = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_category')
df_service = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_service')
df_service_record = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_service_record')
df_category_service = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_category_service')

Dividimos las columnas del df en distintos df segun las tablas de nuestra base de datos.

*Elinamos las filas que tengan duplicada la columna record_id*

In [3]:
df_sin_duplicados = df.drop_duplicates(subset='record_id', keep=False)

In [4]:
df_sin_duplicados.head()

,urls,timestamp,record_id,titles,host_name,property_types,prices_per_night,location,ratings,num_reviews,cleaning_fee,baños,maximum_guests,dormitorios,camas,check_in_hour,check_out_hour,total_hours_checkin
18,https://www.airbnb.es/rooms/113755436612856989...,2024-11-07 10:12:02,1137554366128569898,Haz base en este alojamiento y estarás a un pa...,Steven,Habitación,40.0,"Barcelona, Catalunya, España",4.12,17.0,0.0,NaN,NaN,NaN,2.0,15:00,11:00,9.0
30,https://www.airbnb.es/rooms/124996142740169379...,2024-11-07 10:12:02,1249961427401693793,Disfruten de unos días regeneradores en un amb...,Antonio,otro,128.0,"Barcelona, Catalunya, España",5.00,4.0,0.0,1.0,2.0,2.0,2.0,NaN,11:00,NaN
36,https://www.airbnb.es/rooms/127318511126570438...,2024-11-07 10:12:02,1273185111265704381,Habitación privada para 2 huéspedes en apartam...,Mauricio,Habitación,58.0,"Barcelona, Catalunya, España",0.00,0.0,0.0,NaN,2.0,1.0,NaN,15:00,11:00,9.0
54,https://www.airbnb.es/rooms/117881510800783153...,2024-11-07 10:12:02,1178815108007831530,El apartamento ideal para sumergirte en el alm...,Lodging Apartments,Alojamiento entero,117.0,"Barcelona, Catalunya, España",0.00,2.0,95.0,1.0,2.0,1.0,1.0,15:00,NaN,9.0
57,https://www.airbnb.es/rooms/118896371995828509...,2024-11-07 10:12:02,1188963719958285093,Soy leo y quiero darte la bienvenida a una de ...,Pado,Alojamiento entero,132.0,"Barcelona, Catalunya, España",0.00,0.0,50.0,1.0,2.0,1.0,1.0,15:00,11:00,9.0


In [5]:
df_Hosting = df_sin_duplicados[['record_id','titles','property_types','host_name']]

In [6]:
df_Hosting.head()

,record_id,titles,property_types,host_name
18,1137554366128569898,Haz base en este alojamiento y estarás a un pa...,Habitación,Steven
30,1249961427401693793,Disfruten de unos días regeneradores en un amb...,otro,Antonio
36,1273185111265704381,Habitación privada para 2 huéspedes en apartam...,Habitación,Mauricio
54,1178815108007831530,El apartamento ideal para sumergirte en el alm...,Alojamiento entero,Lodging Apartments
57,1188963719958285093,Soy leo y quiero darte la bienvenida a una de ...,Alojamiento entero,Pado


In [7]:
df_Description_ = df_sin_duplicados[['record_id', 'prices_per_night','check_in_hour','check_out_hour','cleaning_fee','total_hours_checkin','maximum_guests','location','camas','baños','dormitorios']]
df_Description_ = df_Description_.rename(columns={'record_id': 'id'})


In [8]:
df_Ratings = df_sin_duplicados[['record_id','ratings','num_reviews']]

Aqui nos aseguramos de que no haya ningun nan sustituyendolos todos por none (valor que si admite sql).

In [9]:
df_Description_ = df_Description_.replace({np.nan : None}).reset_index(drop = True) #Sustituimos todos los posibles nan por none para poder subirlos
df_Hosting = df_Hosting.replace({np.nan : None}).reset_index(drop = True)
df_Ratings = df_Ratings.replace({np.nan : None}).reset_index(drop = True)

Aqui procedemos a la subida de las distintas tablas.

Hosting

In [11]:
column_names= ['record_id','titles','property_types','host_name']

In [12]:
database = "AIRBNB"
table_name = "Hosting"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Hosting.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Hosting[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 754 filas


Description_

In [13]:
column_names = ['id', 'prices_per_night','check_in_hour','check_out_hour','cleaning_fee','total_hours_checkin','maximum_guests','location','camas','baños','dormitorios']

In [14]:
database = "AIRBNB"
table_name = "Description_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Description_.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Description_[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 754 filas


Ratings

In [15]:
column_names= ['record_id','ratings','num_reviews']

In [ ]:
df_Ratings["record_id"] = df_Ratings["record_id"].apply(lambda x : str(x))

In [28]:
df_Ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   record_id    754 non-null    object 
 1   ratings      754 non-null    float64
 2   num_reviews  754 non-null    float64
dtypes: float64(2), object(1)
memory usage: 17.8+ KB


In [17]:
database = "AIRBNB"
table_name = "Ratings"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names
print(column_names)

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Ratings.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"
print(insert_query)

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Ratings[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

('record_id', 'ratings', 'num_reviews')
INSERT INTO Ratings (record_id, ratings, num_reviews) VALUES (%s, %s, %s)
Añadidas: 754 filas


Services_

In [18]:
column_names = ['service_id', 'service']

In [19]:
database = "AIRBNB"
table_name = "Services_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_service.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_service[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 1143 filas


Services_ids !!!PROBLEMAS!!!

In [10]:
column_names = ['service_id', 'record_id']

In [11]:
df_service_record = df_service_record.rename(columns={'service': 'service_id'})

In [12]:
df_service_record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75056 entries, 0 to 75055
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   service_id  75056 non-null  int64
 1   record_id   75056 non-null  int64
dtypes: int64(2)
memory usage: 1.1 MB


In [13]:
df_service_record["record_id"] = df_service_record["record_id"].apply(lambda x : str(x))

In [14]:
df_service_record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75056 entries, 0 to 75055
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   service_id  75056 non-null  int64 
 1   record_id   75056 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [15]:
database = "AIRBNB"
table_name = "Services_ids"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_service_record.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_service_record[list(column_names)].values] 

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`airbnb`.`services_ids`, CONSTRAINT `services_ids_ibfk_2` FOREIGN KEY (`record_id`) REFERENCES `hosting` (`record_id`))

Category

In [16]:
column_names = ['category_id', 'category']

In [17]:
database = "AIRBNB"
table_name = "Category"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_category.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_category[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 13 filas


Category_ids  !!!PROBLEMAS!!!!

In [18]:
column_names = ['category_id', 'service_id']

In [19]:
df_category_service = df_category_service.rename(columns={'category': 'category_id'})
df_category_service = df_category_service.rename(columns={'service': 'service_id'})

In [20]:
database = "AIRBNB"
table_name = "Category_ids"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_category_service.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_category_service[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

InterfaceError: Failed executing the operation; Python type numpy.int64 cannot be converted

# Extracción de datos

In [ ]:
def extraer_datos_mysql(host, user, password, database, query):
    # Conectarse a la base de datos
    db = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Crear cursor para ejecutar la consulta
    cursor = db.cursor()

    # Ejecutar la consulta
    cursor.execute(query)
    
    # Obtener los nombres de las columnas
    column_names = cursor.column_names
    print("Nombres de las columnas:", column_names)

    # Guardar y mostrar los datos de la tabla
    data = cursor.fetchall()
    for fila in data:
        print(fila)

    # Cerrar el cursor y la conexión
    cursor.close()
    db.close()


Hosting

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Hosting;"

extraer_datos_mysql(host, user, password, database, query)

Description_

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Description_;"

extraer_datos_mysql(host, user, password, database, query)

Ratings

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Ratings;"

extraer_datos_mysql(host, user, password, database, query)

Services

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Services;"

extraer_datos_mysql(host, user, password, database, query)

Category

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Category;"

extraer_datos_mysql(host, user, password, database, query)